## Atterrissage et décollage TOULOUSE

In [5]:
import pandas as pd
import numpy as np
import ast
import json
from openap import FlightPhase
import matplotlib.pyplot as plt
from openap import Emission, FuelFlow, prop
from mpl_toolkits.mplot3d import Axes3D

data_new_na = pd.read_csv('../Data/ADSB_cleaned_11_2022/data_join_25-11.csv')

list_time = data_new_na.timestamp_s
list_type = data_new_na.type_avion
list_tas = data_new_na.TAS
list_alt = data_new_na.altitude
list_distance = data_new_na.distance_aero
list_rate = data_new_na.vertical_rate

In [133]:
def test_segment(labels, ts, alt, spd, roc):

    phasecolors = {
        'GND': 'black',     #ground
        'CL': 'green',      #climb
        'DE': 'blue',       #descend
        'LVL': 'cyan',      #level
        'CR': 'purple',     #cruise (vitesse cste ou alt cste)
        'NA': 'red'         #not available
    }

    colors = [phasecolors[lbl] for lbl in labels]

    plt.subplot(311)
    plt.scatter(ts, alt, marker='.', c=colors, lw=0)
    plt.ylabel('altitude (ft)')

    plt.subplot(312)
    plt.scatter(ts, spd, marker='.', c=colors, lw=0)
    plt.ylabel('speed (kt)')

    plt.subplot(313)
    plt.scatter(ts, roc, marker='.', c=colors, lw=0)
    plt.ylabel('roc (fpm)')

    plt.show()

list_decollage = []
list_atterrissage = []
for nb in range(len(list_alt)) :
    
    ts = np.array(ast.literal_eval(list_time[nb]))
    spd = np.array(ast.literal_eval(list_tas[nb]))
    alt = np.array(ast.literal_eval(list_alt[nb]))
    distance = np.array(ast.literal_eval(list_distance[nb]))
    roc = np.array(ast.literal_eval(list_rate[nb]))
    tas_, alt_ = np.meshgrid(spd, alt)

    distance_index = np.argmin(distance)
    distance_min = distance[distance_index]
    sueil = 30    #depend des aeroports alentours
    dict = {}
    if  (distance_min < sueil) : 

        # Recherche des indices des changements de signe (décollage ou atterrissage)
        sign_change = np.where(np.diff(np.sign(roc)))[0] + 1

        # Séparation des intervalles de décollage et d'atterrissage
        intervals_roc = np.split(roc, sign_change)
        intervals_time = np.split(ts, sign_change)

        # Decider si atterrissage ou décollage
        dict["DECOLLAGE"] = []
        dict["ATTERRISSAGE"] = []
        for i, interval in enumerate(intervals_roc):
            try :
                if (interval[0] > 0) & (len(interval) >=20  )& (max(interval) >= 1000):
                    dict["DECOLLAGE"].append(intervals_time[i].tolist())
                elif (interval[0] < 0) & (len(interval) >=20 )& (min(interval) <= -1000) : 
                    dict["ATTERRISSAGE"].append(intervals_time[i].tolist())
            except : 
                print("liste vide")


        list_decollage.append(json.dumps(dict["DECOLLAGE"]))
        list_atterrissage.append(json.dumps(dict["ATTERRISSAGE"]))

    else :
        list_decollage.append("[]")       #Pas sur Toulouse
        list_atterrissage.append("[]")


data_new_na.loc[:, 'DECOLLAGE'] = list_decollage
data_new_na.loc[:, 'ATTERRISSAGE'] = list_atterrissage

data_new_na
        

,Unnamed: 0,tale_nb,type_avion,jet_prive,serial_nb,Flight_ID,timestamp_s,datetime,altitude,longitude,latitude,distance_aero,TAS,vertical_rate,indice_sonore,DECOLLAGE,ATTERRISSAGE
0,020145,CN-NMO,A320,Air Arabia,Unknown,MAC341H,"[1669283573, 1669276498, 1669276497, 166927649...","[Timestamp('2022-11-24 09:52:53'), Timestamp('...","[28825, 24975, 24975, 24975, 25025, 25025, 250...","[5.154353550502232, 4.839281354631696, 4.83928...","[44.615203857421875, 44.66908264160156, 44.669...","[321.4937044275735, 300.1193717943871, 300.119...","[395, 358, 358, 359, 359, 360, 360, 360, 361, ...","[1472, -1344, -1344, -1344, -1408, -1408, -140...",62.979379,[],[]
1,020145,CN-NMO,A320,Air Arabia,Unknown,MAC342,"[1669300860, 1669300859, 1669286110, 166928610...","[Timestamp('2022-11-24 14:41:00'), Timestamp('...","[38000, 38000, 35000, 35000, 35000, 35000, 350...","[-1.6514622887899577, -1.6514622887899577, 1.6...","[41.6541510113215, 41.6541510113215, 41.005462...","[330.43912615087027, 330.43912615087027, 292.7...","[508, 508, 389, 389, 388, 388, 388, 388, 388, ...","[0, 0, 64, 64, -128, -128, -128, -128, -128, -...",66.137470,[],[]
2,020145,CN-NMO,A320,Air Arabia,Unknown,MAC357E,"[1669302848, 1669302838, 1669302833, 166930283...","[Timestamp('2022-11-24 15:14:08'), Timestamp('...","[425, 425, 425, 425, 425, 425, 425, 425, 425, ...","[1.3541303362165178, 1.3541303362165178, 1.354...","[43.636298745365465, 43.636298745365465, 43.63...","[1.3344598919222888, 1.3344598919222888, 1.334...","[2, 2, 2, 2, 2, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, ...","[-64, -64, -64, -64, -64, -64, -64, -64, -64, ...",78.938155,[],"[[1669302848, 1669302838, 1669302833, 16693028..."
3,020145,CN-NMO,A320,Air Arabia,Unknown,MAC358A,"[1669308262, 1669308257, 1669308256, 166930825...","[Timestamp('2022-11-24 16:44:22'), Timestamp('...","[37000, 37000, 37000, 37025, 37000, 37000, 370...","[-1.2320656000181316, -1.2320656000181316, -1....","[41.43847449351165, 41.43847449351165, 41.4397...","[323.5659945088089, 323.5659945088089, 323.390...","[375, 374, 374, 374, 374, 374, 373, 371, 372, ...","[0, 0, 0, 0, 0, 0, -64, 0, -64, -64, -64, -64,...",77.244746,"[[1669307808, 1669307807, 1669307806, 16693078...",[]
4,02a1b3,TS-INT,A320,Unknown,Unknown,LBT514,"[1669306536, 1669306535, 1669306534, 166930653...","[Timestamp('2022-11-24 16:15:36'), Timestamp('...","[400, 400, 400, 400, 400, 400, 400, 400, 400, ...","[1.3538094454033431, 1.3538094454033431, 1.353...","[43.63655090332031, 43.63655090332031, 43.6365...","[1.3718103523391902, 1.3718103523391902, 1.371...","[0, 0, 0, 1, 0, 1, 2, 2, 1, 2, 2, 2, 2, 3, 3, ...","[-64, -64, -64, -64, -64, -64, -64, -64, -64, ...",79.385153,[],"[[1669306536, 1669306535, 1669306534, 16693065..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,4ca7b7,EI-EGB,B738,Ryanair Designated Activity Company,38491,RYR4SL,"[1669275055, 1669275053, 1669275052, 166927505...","[Timestamp('2022-11-24 07:30:55'), Timestamp('...","[325, 325, 325, 325, 325, 325, 350, 350, 400, ...","[1.353607177734375, 1.353607177734375, 1.35023...","[43.636764267743644, 43.636764267743644, 43.63...","[1.3989561352155446, 1.3989561352155446, 1.834...","[117, 117, 119, 123, 135, 135, 139, 139, 139, ...","[-64, -64, -128, -64, -192, -128, -448, -512, ...",78.808155,[],"[[1669275055, 1669275053, 1669275052, 16692750..."
252,4ca7b7,EI-EGB,B738,Ryanair Designated Activity Company,38491,RYR6FP,"[1669300531, 1669300530, 1669300528, 166930052...","[Timestamp('2022-11-24 14:35:31'), Timestamp('...","[35000, 35000, 35000, 35000, 35575, 35575, 355...","[5.020621163504464, 5.020621163504464, 5.02062...","[42.87484379137977, 42.87484379137977, 42.8748...","[307.4863924155051, 307.4863924155051, 307.486...","[378, 378, 378, 378, 378, 495, 495, 525, 526, ...","[0, 0, 0, 0, 0, -2176, -2176, 0, 0, 0, 0, 64, ...",70.978956,[],[]
253,4cafc2,EI-GXG,B138,Unknown,Unknown,RYR1042,"[1669316387, 1669316386, 1669316385, 166931638...","[Timestamp('2022-11-24 18:59:47'), Timestamp('..."

In [134]:
#data_new = data_new_na.filter(data_new_na['ATTERRISSAGE'] != '[]')
data_new = data_new_na.loc[(data_new_na['ATTERRISSAGE'] != '[]') | (data_new_na['DECOLLAGE'] != '[]')]
data_new

,Unnamed: 0,tale_nb,type_avion,jet_prive,serial_nb,Flight_ID,timestamp_s,datetime,altitude,longitude,latitude,distance_aero,TAS,vertical_rate,indice_sonore,DECOLLAGE,ATTERRISSAGE
2,020145,CN-NMO,A320,Air Arabia,Unknown,MAC357E,"[1669302848, 1669302838, 1669302833, 166930283...","[Timestamp('2022-11-24 15:14:08'), Timestamp('...","[425, 425, 425, 425, 425, 425, 425, 425, 425, ...","[1.3541303362165178, 1.3541303362165178, 1.354...","[43.636298745365465, 43.636298745365465, 43.63...","[1.3344598919222888, 1.3344598919222888, 1.334...","[2, 2, 2, 2, 2, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, ...","[-64, -64, -64, -64, -64, -64, -64, -64, -64, ...",78.938155,[],"[[1669302848, 1669302838, 1669302833, 16693028..."
3,020145,CN-NMO,A320,Air Arabia,Unknown,MAC358A,"[1669308262, 1669308257, 1669308256, 166930825...","[Timestamp('2022-11-24 16:44:22'), Timestamp('...","[37000, 37000, 37000, 37025, 37000, 37000, 370...","[-1.2320656000181316, -1.2320656000181316, -1....","[41.43847449351165, 41.43847449351165, 41.4397...","[323.5659945088089, 323.5659945088089, 323.390...","[375, 374, 374, 374, 374, 374, 373, 371, 372, ...","[0, 0, 0, 0, 0, 0, -64, 0, -64, -64, -64, -64,...",77.244746,"[[1669307808, 1669307807, 1669307806, 16693078...",[]
4,02a1b3,TS-INT,A320,Unknown,Unknown,LBT514,"[1669306536, 1669306535, 1669306534, 166930653...","[Timestamp('2022-11-24 16:15:36'), Timestamp('...","[400, 400, 400, 400, 400, 400, 400, 400, 400, ...","[1.3538094454033431, 1.3538094454033431, 1.353...","[43.63655090332031, 43.63655090332031, 43.6365...","[1.3718103523391902, 1.3718103523391902, 1.371...","[0, 0, 0, 1, 0, 1, 2, 2, 1, 2, 2, 2, 2, 3, 3, ...","[-64, -64, -64, -64, -64, -64, -64, -64, -64, ...",79.385153,[],"[[1669306536, 1669306535, 1669306534, 16693065..."
5,02a1b3,TS-INT,A320,Unknown,Unknown,LBT515,"[1669311878, 1669311873, 1669311872, 166931187...","[Timestamp('2022-11-24 17:44:38'), Timestamp('...","[36000, 36000, 36000, 36000, 36000, 36000, 360...","[3.797884209211483, 3.7847900390625, 3.7847900...","[41.55094470007945, 41.56123352050781, 41.5612...","[304.9127850078826, 303.3362101055605, 303.336...","[493, 493, 493, 492, 492, 492, 492, 492, 492, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",77.506088,"[[1669311748, 1669311747, 1669311746, 16693117...",[]
6,0a0022,7T-VJP,B738,Air Algerie,30208,DAH1076,"[1669283259, 1669283258, 1669283257, 166928325...","[Timestamp('2022-11-24 09:47:39'), Timestamp('...","[325, 325, 325, 325, 325, 325, 325, 325, 325, ...","[1.3572038922991072, 1.3572038922991072, 1.357...","[43.633319402145126, 43.633319402145126, 43.63...","[0.9485907998018277, 0.9485907998018277, 0.948...","[0, 1, 2, 2, 2, 3, 3, 5, 5, 5, 5, 5, 6, 6, 6, ...","[-64, -64, -64, -64, -64, -64, -64, -64, -64, ...",79.838038,[],"[[1669283259, 1669283258, 1669283257, 16692832..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,4ca7b7,EI-EGB,B738,Ryanair Designated Activity Company,38491,RYR1GB,"[1669289091, 1669289090, 1669289089, 166928908...","[Timestamp('2022-11-24 11:24:51'), Timestamp('...","[38000, 38000, 38000, 38000, 38000, 38000, 380...","[-3.058798146802303, -3.058798146802303, -3.06...","[42.39279536877648, 42.39279536877648, 42.3917...","[385.2090036802727, 385.2090036802727, 385.482...","[528, 528, 528, 528, 528, 528, 528, 358, 358, ...","[0, 0, 0, 0, 0, 0, 64, 0, 0, 0, 0, 0, 0, 0, 0,...",77.747888,"[[1669280064, 1669280063, 1669280062, 16692800...",[]
251,4ca7b7,EI-EGB,B738,Ryanair Designated Activity Company,38491,RYR4SL,"[1669275055, 1669275053, 1669275052, 166927505...","[Timestamp('2022-11-24 07:30:55'), Timestamp('...","[325, 325, 325, 325, 325, 325, 350, 350, 400, ...","[1.353607177734375, 1.353607177734375, 1.35023...","[43.636764267743644, 43.636764267743644, 43.63...","[1.3989561352155446, 1.3989561352155446, 1.834...","[117, 117, 119, 123, 135, 135, 139, 139, 139, ...","[-64, -64, -128, -64, -192, -128, -448, -512, ...",78.808155,[],"[[1669275055, 1669275053, 1669275052, 16692750..."
253,4ca

### Ajouter la colonne des ages des avions et nombre de passagers dans la bdd atterrissage

In [75]:
import requests
import html_to_json
import json

In [76]:
# return None if serial number or tale number are not found
# returns airecraft age and the number of passenger in normal conditions (tale number and serial number exists) 

def getAge_bis(tale_nb,serial_nb) :
    airecraft_ages = []
    url = f"https://www.airport-data.com/aircraft/{tale_nb}.html"

    response = requests.get(url)
    data = response.text
    output = html_to_json.convert(data)
    f = open("answer.json", "w")
    f.write(json.dumps(output))
    f.close()
    try :
        aircraft_age_table = output["html"][0]["body"][0]["div"][0]["div"][3]["div"][1]["div"]
        for i in range(len(aircraft_age_table)) :
            num_airecraft = i
            airecraft_age = ""
            dib_numb = len(aircraft_age_table[i]["div"]) - 1
            passager_seat_ind = 5
            airecraft_age_from_html = aircraft_age_table[num_airecraft]["div"][dib_numb]["table"][0]["tr"][2]["td"][1]["_value"]
            serial_numb = aircraft_age_table[num_airecraft]["div"][dib_numb]["table"][0]["tr"][3]["td"][1]["_value"]
            check = aircraft_age_table[num_airecraft]["div"][dib_numb]["table"][0]["tr"][5]["td"][0]["b"][0]["_value"]
            if check == "Aircraft Type:"  :
                passager_seat_ind = 6
            nombre_passagers = aircraft_age_table[num_airecraft]["div"][dib_numb]["table"][0]["tr"][passager_seat_ind]["td"][1]["_value"]
            serial_numbers = serial_numb.split("/")
            airecraft_ages.append({"serial_nb" : serial_numb, "age" : airecraft_age_from_html})
            if serial_nb in serial_numbers or serial_nb == serial_numb : 
                print(f"the airecraft age is {airecraft_age_from_html} et le nombre de passagers est {nombre_passagers}")
                airecraft_age = airecraft_age_from_html
        if (airecraft_age == "") :
            return None, None
        return airecraft_age , nombre_passagers
    except :
        print("not found")
        return None, None
# example
print(getAge_bis("Z-WPA","23677"))

the airecraft age is 1986 et le nombre de passagers est 100
('1986', '100')


In [135]:
age_list = []
nombre_passagers_list = []

for i in data_new.index : 
    tale_nb = data_new['tale_nb'][i]
    serial_nb = data_new['serial_nb'][i]
    airecraft_age , nombre_passagers = getAge_bis(tale_nb,serial_nb)
    age_list.append(airecraft_age)
    nombre_passagers_list.append(nombre_passagers)

data_new.loc[:, 'age_avion'] = age_list
data_new.loc[:, 'nombre_passagers'] = nombre_passagers_list

the airecraft age is 2001 et le nombre de passagers est N/A
the airecraft age is 2001 et le nombre de passagers est N/A
the airecraft age is 2010 et le nombre de passagers est N/A
the airecraft age is 2010 et le nombre de passagers est N/A
the airecraft age is 2010 et le nombre de passagers est 100
the airecraft age is 2010 et le nombre de passagers est 100
the airecraft age is 2010 et le nombre de passagers est 100
the airecraft age is 2016 et le nombre de passagers est 100
the airecraft age is 2016 et le nombre de passagers est 100
the airecraft age is 2017 et le nombre de passagers est 100
the airecraft age is 2017 et le nombre de passagers est 70
the airecraft age is 2014 et le nombre de passagers est N/A
the airecraft age is 2001 et le nombre de passagers est N/A
the airecraft age is 1995 et le nombre de passagers est N/A
the airecraft age is 1995 et le nombre de passagers est N/A
the airecraft age is 1995 et le nombre de passagers est N/A
the airecraft age is 1995 et le nombre de

C:\Users\chell\AppData\Local\Temp\ipykernel_115252\1770918280.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_new.loc[:, 'age_avion'] = age_list
C:\Users\chell\AppData\Local\Temp\ipykernel_115252\1770918280.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_new.loc[:, 'nombre_passagers'] = nombre_passagers_list


In [136]:
data_new

,Unnamed: 0,tale_nb,type_avion,jet_prive,serial_nb,Flight_ID,timestamp_s,datetime,altitude,longitude,latitude,distance_aero,TAS,vertical_rate,indice_sonore,DECOLLAGE,ATTERRISSAGE,age_avion,nombre_passagers
2,020145,CN-NMO,A320,Air Arabia,Unknown,MAC357E,"[1669302848, 1669302838, 1669302833, 166930283...","[Timestamp('2022-11-24 15:14:08'), Timestamp('...","[425, 425, 425, 425, 425, 425, 425, 425, 425, ...","[1.3541303362165178, 1.3541303362165178, 1.354...","[43.636298745365465, 43.636298745365465, 43.63...","[1.3344598919222888, 1.3344598919222888, 1.334...","[2, 2, 2, 2, 2, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, ...","[-64, -64, -64, -64, -64, -64, -64, -64, -64, ...",78.938155,[],"[[1669302848, 1669302838, 1669302833, 16693028...",None,None
3,020145,CN-NMO,A320,Air Arabia,Unknown,MAC358A,"[1669308262, 1669308257, 1669308256, 166930825...","[Timestamp('2022-11-24 16:44:22'), Timestamp('...","[37000, 37000, 37000, 37025, 37000, 37000, 370...","[-1.2320656000181316, -1.2320656000181316, -1....","[41.43847449351165, 41.43847449351165, 41.4397...","[323.5659945088089, 323.5659945088089, 323.390...","[375, 374, 374, 374, 374, 374, 373, 371, 372, ...","[0, 0, 0, 0, 0, 0, -64, 0, -64, -64, -64, -64,...",77.244746,"[[1669307808, 1669307807, 1669307806, 16693078...",[],None,None
4,02a1b3,TS-INT,A320,Unknown,Unknown,LBT514,"[1669306536, 1669306535, 1669306534, 166930653...","[Timestamp('2022-11-24 16:15:36'), Timestamp('...","[400, 400, 400, 400, 400, 400, 400, 400, 400, ...","[1.3538094454033431, 1.3538094454033431, 1.353...","[43.63655090332031, 43.63655090332031, 43.6365...","[1.3718103523391902, 1.3718103523391902, 1.371...","[0, 0, 0, 1, 0, 1, 2, 2, 1, 2, 2, 2, 2, 3, 3, ...","[-64, -64, -64, -64, -64, -64, -64, -64, -64, ...",79.385153,[],"[[1669306536, 1669306535, 1669306534, 16693065...",None,None
5,02a1b3,TS-INT,A320,Unknown,Unknown,LBT515,"[1669311878, 1669311873, 1669311872, 166931187...","[Timestamp('2022-11-24 17:44:38'), Timestamp('...","[36000, 36000, 36000, 36000, 36000, 36000, 360...","[3.797884209211483, 3.7847900390625, 3.7847900...","[41.55094470007945, 41.56123352050781, 41.5612...","[304.9127850078826, 303.3362101055605, 303.336...","[493, 493, 493, 492, 492, 492, 492, 492, 492, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",77.506088,"[[1669311748, 1669311747, 1669311746, 16693117...",[],None,None
6,0a0022,7T-VJP,B738,Air Algerie,30208,DAH1076,"[1669283259, 1669283258, 1669283257, 166928325...","[Timestamp('2022-11-24 09:47:39'), Timestamp('...","[325, 325, 325, 325, 325, 325, 325, 325, 325, ...","[1.3572038922991072, 1.3572038922991072, 1.357...","[43.633319402145126, 43.633319402145126, 43.63...","[0.9485907998018277, 0.9485907998018277, 0.948...","[0, 1, 2, 2, 2, 3, 3, 5, 5, 5, 5, 5, 6, 6, 6, ...","[-64, -64, -64, -64, -64, -64, -64, -64, -64, ...",79.838038,[],"[[1669283259, 1669283258, 1669283257, 16692832...",2001,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,4ca7b7,EI-EGB,B738,Ryanair Designated Activity Company,38491,RYR1GB,"[1669289091, 1669289090, 1669289089, 166928908...","[Timestamp('2022-11-24 11:24:51'), Timestamp('...","[38000, 38000, 38000, 38000, 38000, 38000, 380...","[-3.058798146802303, -3.058798146802303, -3.06...","[42.39279536877648, 42.39279536877648, 42.3917...","[385.2090036802727, 385.2090036802727, 385.482...","[528, 528, 528, 528, 528, 528, 528, 358, 358, ...","[0, 0, 0, 0, 0, 0, 64, 0, 0, 0, 0, 0, 0, 0, 0,...",77.747888,"[[1669280064, 1669280063, 1669280062, 16692800...",[],2009,189
251,4ca7b7,EI-EGB,B738,Ryanair Designated Activity Company,38491,RYR4SL,"[1669275055, 1669275053, 1669275052, 166927505...","[Timestamp('2022-11-24 07:30:55'), Timestamp('...","[325, 325, 325, 325, 325, 325, 350, 350, 400, ...","[1.353607177734375, 1.353607177734375, 1.35023...","[43.636764267743644, 43.636764267743644, 43.63...","[1.3989561352155446, 1.3989561352155446, 1.834...","[117, 117, 119, 123, 135, 135, 139, 139, 139, ...","[-64, -64, -128, -64, -192, -1

In [137]:
#data_new.to_csv("data_toulouse_29_09.csv")
data_new.to_csv("../Data/ADSB_cleaned_11_2022/data_toulouse_25_11.csv")

In [138]:
data_atterrissage = data_new[data_new['ATTERRISSAGE'] != '[]']
data_atterrissage2 = data_atterrissage.drop(columns=['DECOLLAGE'])
data_decollage = data_new[data_new['DECOLLAGE'] != '[]']
data_decollage2 = data_decollage.drop(columns=['ATTERRISSAGE'])

In [139]:
data_atterrissage2.to_csv('../Data/ADSB_cleaned_11_2022/data_toulouse_atterrisage_25-11.csv')
data_decollage2.to_csv('../Data/ADSB_cleaned_11_2022/data_toulouse_decollage_25-11.csv')